<a href="https://colab.research.google.com/github/Alisoltan82/Mammogram-app/blob/main/BREAST_CANCER_CLSS_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'king-abdulaziz-university-mammogram-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1428914%2F3243177%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240213%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240213T095526Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da24ad3bb6bb6ae754f97fc669e87a425366cd0cb9316e0c04a5b93c48b9844202c5f07e2e7a8d32f2c0c3a53e76d21ff2cf57b3b693811a075d9885c1fc790fd25f99ce079aa4c3913ef44a91a5787f2d2eca4d64d4bba56fab3a94cf00980abefee832e99141881f11023505e1f2bfe3f5a9315c89d845907a12a9bf4825fdfbc8d4e41def2f36857f0159b75960c0d2c50f54e3793568c10f68384d5e41814c1d091363c3df5bc948b044a7030cf54bdb205697c8fafa4a89a9fb9388c7d0d2779584a6f3a87d2eadea2f18ea5f7f1dbf55cc448f3454eafd428527277499b3457079dfb6c9247c8cfbddd78bcc13875a121385a344f55020555cc14e1e12c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install monai

In [ ]:
from monai.apps.tcia import TCIA_LABEL_DICT
from monai.config import print_config
from monai.networks.nets import UNet , DenseNet , EfficientNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric , get_confusion_matrix
from monai.losses import DiceLoss , DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch , ArrayDataset
from monai.apps import TciaDataset
from monai.config import print_config , KeysCollection
from monai.utils import first , set_determinism
from monai.transforms import (
    Compose,
    LoadImage,
    LoadImaged,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    ToTensor,
    ToTensord,
    ScaleIntensityRange,
    ScaleIntensityRanged,
    ThresholdIntensity,
    ThresholdIntensityd,
    SaveImaged,
    Spacingd,
    CropForegroundd,
    Orientationd,
    AsDiscrete,
    RandCropByPosNegLabeld,
    DivisiblePadd,
    Resized,
    RandFlipd,
    RandRotate90d,
    RandShiftIntensityd




)

print_config()

In [ ]:
from glob import glob
import os
import cv2
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
ROOT_PATH = Path('/kaggle/input/king-abdulaziz-university-mammogram-dataset/')
SAVE_PATH = Path('/kaggle/working/output')
img_path_list = sorted(glob(os.path.join(ROOT_PATH/'*'/'*'/'*.jpg')))
len(img_path_list)

In [ ]:
#building dataframe via img_folders

data = pd.DataFrame()
data['path'] = img_path_list
data

In [ ]:
data['filename'] = data['path'].str.split('/').str[-1]
data['date'] = data['filename'].str.split('_').str[0]
data['patientid'] = data['filename'].str.split('_').str[1]
data['mode'] = data['filename'].str.split('_').str[2]
data['position'] = data['filename'].str.split('_').str[-1].str.split('.').str[0]
data['label'] = data['path'].str.split('/').str[-3]
data

In [ ]:
#images visualization
fig , axis = plt.subplots(4,4,figsize = (10,15))
c = 0

for i in range(4):
    for j in range(4):
        img_path = data['path'].iloc[c]
        img = cv2.imread(img_path , cv2.IMREAD_GRAYSCALE)

        label = data['label'].iloc[c]
        mode = data['mode'].iloc[c]

        axis[i][j].imshow(img , cmap = 'bone')
        axis[i][j].set_title(f'{label}\n{mode}')
        c+=100

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

encoder = LabelEncoder()
data['y'] = encoder.fit_transform(data['label'])


x = data['path'][:-10]
y = data['y'][:-10]

x_test = data['path'][-10:]
y_test = data['y'][-10:]

x_train,x_test , y_train, y_test = train_test_split(x,y , test_size = 0.3 , random_state = 42 ,shuffle = True)

len(x_train) , len(y_train) , len(x_test) , len(y_test)

In [ ]:
test = cv2.imread(data['path'][0] , cv2.IMREAD_GRAYSCALE)
print(np.max(test)) , print(np.min(test))
test_s = test/255
print(np.max(test_s)) , print(np.min(test_s) , test_s.shape)
nn = np.sum(test_s)/ 224**2
print(nn)

In [ ]:
SAVE_DIR = Path('/kaggle/working/')

from tqdm.auto import tqdm

sums , sums_squared = 0,0

for c, patient in enumerate(tqdm(x_train)):
    img_path = x_train.iloc[c]
    img = cv2.imread(img_path , cv2.IMREAD_GRAYSCALE) /255 #scaling
    img_r = cv2.resize(img , (224,224)).astype(np.float16)
    label = y_train.iloc[c]


    train_folder = 'train'
    save_path = SAVE_DIR/ train_folder / str(label)
    save_path.mkdir(parents = True , exist_ok = True)
    np.save(save_path/str(c).zfill(3) , img_r)

    normalizer = 224*224 #new image dims

    sums += np.sum(img_r)/normalizer
    sums_squared += (img_r **2).sum() / normalizer

In [ ]:
for c, patient in enumerate(tqdm(x_test)):
    img_path = x_test.iloc[c]
    img = cv2.imread(img_path , cv2.IMREAD_GRAYSCALE) /255 #scaling
    img_r = cv2.resize(img , (224,224)).astype(np.float16)
    label = y_test.iloc[c]


    test_folder = 'val'
    save_path = SAVE_DIR/ test_folder / str(label)
    save_path.mkdir(parents = True , exist_ok = True)
    np.save(save_path/str(c).zfill(3) , img_r)

In [ ]:
mean = sums / len(x_train)
print(f'train images mean = {mean}')

std = np.sqrt((sums_squared/len(x_train)) - mean**2)
print(f'train images standard deviation = {std}')

In [ ]:
mean.dtype

In [ ]:
import torch
from torch import nn , optim, Tensor, manual_seed, argmax
import torchvision
from torchvision import transforms
import torchmetrics
import torch.nn.functional as F

torch.__version__

In [ ]:
def load_file(path):
    return np.load(path).astype(np.float32)


train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = mean , std = std ),
    transforms.RandomAffine(degrees = (-5,5) , translate = (0, 0.05) , scale = (0.9 , 1.1) ),
    transforms.RandomResizedCrop(size = (224,224) , scale = (0.5 , 1.0), antialias = True)

])


val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = mean , std = std),

])

In [ ]:
training = glob(os.path.join('/kaggle/working/train' , '*' ,'*.npy') , recursive = True)
testing = glob(os.path.join('/kaggle/working/val' , '*' ,'*.npy'), recursive = True)
len(training) , len(testing)

In [ ]:
train_dataset = torchvision.datasets.DatasetFolder('/kaggle/working/train' ,
                                                   loader = load_file ,
                                                   extensions = 'npy' ,
                                                   transform = train_transforms)

val_dataset = torchvision.datasets.DatasetFolder('/kaggle/working/val' ,
                                                   loader = load_file ,
                                                   extensions = 'npy' ,
                                                   transform = val_transforms)

In [ ]:
fig , axis = plt.subplots(3,3,figsize = (9,10))
for i in range(3):
    for j in range(3):
        random_idx = np.random.randint(0 , 300)
        img , label = train_dataset[random_idx]
        axis[i][j].imshow(img[0] , cmap = 'bone')
        axis[i][j].set_title(label)
plt.show();

In [ ]:
#data_loader for datasets
batch_size = 32
num_workers = os.cpu_count()

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = batch_size,
                                          num_workers = num_workers,
                                          shuffle = True)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                          batch_size = batch_size,
                                          num_workers = num_workers,
                                          shuffle = False)

In [ ]:
#checking data imbalance
np.unique(train_dataset.targets , return_counts = True)

In [ ]:
train_features_batch , train_labels_batch = next(iter(train_loader))
train_features_batch.shape , train_labels_batch.shape , train_labels_batch.dtype

In [ ]:
from monai.networks.nets import EfficientNetBN
model = EfficientNetBN("efficientnet-b5",
                       pretrained=True,
                       spatial_dims=2,
                       in_channels=1,
                       num_classes=4).to(device)

# outputs = model(inputs)

In [ ]:
#Model
device = torch.device("cuda")
# model = EfficientNet(blocks_args_str=['r1_k3_s11_e1_i32_o4_se0.25']
#     ,spatial_dims=2,
#     in_channels=1,
#     num_classes = 4,
#     image_size = 224,
#     norm=Norm.BATCH,
# ).to(device)

loss_function = nn.CrossEntropyLoss(weight = torch.tensor([1/1306 , 1/267 , 1/76 , 1/8]) ).to(device)
optimizer = torch.optim.Adam(model.parameters(), 1e-3)
Accuracy = torchmetrics.Accuracy(task='multiclass',
                                     num_classes=4)

In [ ]:
max_epochs = 100
val_interval = 1
best_metric = -1
best_metric_epoch = -1
train_losses = []
val_losses = []
metric_values = []
train_correct = []
val_correct = []


for epoch in range(max_epochs):

    trn_corr , val_corr = 0,0

    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()

    step = 0
    for batch,(x,y) in enumerate(train_loader):
        total_loss = 0
        step += 1
        x, y = (
            x.to(device),
            y.to(device),
        )
        logits = model(x)

        loss = loss_function(logits, y)
        y_pred = argmax(torch.softmax(logits , dim = 1) , dim = 1)
        batch_corr = (y_pred == y).sum()
        trn_corr += batch_corr

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()


#         if step % 2 == 0:
#             print(f'epoch: {epoch +1} | BATCH:{batch} | LOSS: {loss.item()}')
        #print(f'MEAN_loss: {mean_loss/len(train_loader)}')

    #epoch_loss /= step
    train_losses.append(total_loss)
    train_correct.append(trn_corr)
    #print(f'epoch: {epoch +1} | mean loss: {total_loss}')
    np.save(os.path.join(SAVE_DIR , 'train_losses.npy'),total_loss)


    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.inference_mode():
            val_loss,val_acc = 0,0
            for batch,(x,y) in enumerate(val_loader):
                x, y = (
                    x.to(device),
                    y.to(device),
                )

                logits = model(x)
                loss = loss_function(logits, y)
                y_pred = argmax(torch.softmax(logits, dim = 1) , dim = 1)

                val_loss += loss.item()
                val_corr += (y_pred == y).sum()


            print(f'epoch: {epoch} | \n val_corr: {val_corr} out of {len(val_dataset)} ACC: {val_corr/len(val_dataset) * 100}')

        val_losses.append(val_loss)
        val_correct.append(val_corr)
        np.save(os.path.join(SAVE_DIR , 'val_losses.npy'),val_losses)
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, "breast_metric_B7_model.pth"))



In [ ]:
plt.plot(train_losses , label = 'train')
plt.plot(val_losses , label = 'val')
plt.legend()

In [ ]:
train_correct

In [ ]:
plt.plot([t.cpu().detach().numpy()/len(train_dataset) for t in train_correct] , label = 'train_corr')
plt.plot([t.cpu().detach().numpy()/len(val_dataset) for t in val_correct] , label = 'val_corr')
plt.legend()

In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [i + 1 for i in range(len(train_losses))]
y = train_losses
plt.xlabel("epoch")
plt.plot(x, y)
# plt.subplot(1, 2, 2)
# plt.title("Val Mean Dice")
# x = [val_interval * (i + 1) for i in range(len(val_loss))]
# y = val_loss
# plt.xlabel("epoch")
# plt.plot(x, y)
# plt.show()